# Io python

In this lesson you will learn about the IO functions of Python.
This will enable you to read and write files as well as 
read and write to serial devices such as the secure bootloader 
you are designing during this course. A series of challenges
follow which will require you to read Python documentation
as well as other online resources. Good luck!

### Challenge Name: echo_int (/embsec/io_python/echo_int)


    1. Read a big-endian short from the serial device
    2. Increment the integer by 1
    3. Send the integer as a big-endian short back to the serial device

Resources:

<https://en.wikipedia.org/wiki/Endianness>

<https://en.wikipedia.org/wiki/Integer_%28computer_science%29>

<https://docs.python.org/3/library/struct.html>




In [3]:
from embsec import Serial
import struct 

def echo_int():
    ser = Serial("/embsec/io_python/echo_int")
    file = struct.unpack('>h',ser.read(2))
    print(file)
    inp = 1 + file[0]
    ser.write(struct.pack('>h', inp))
    return ser.read_until()
echo_int()


(176,)


b'embsec{echo_int_692aa2777036fd17}\n'

### Challenge Name: send_file (/embsec/io_python/send_file)



    1. Read 'file.bin' from your local system
    2. Calculate the size
    3. Send the size as a little-endian short
    4. Send the file to serial device

    The serial device expects a little-endian short indicating the size of the
    incoming data and then size bytes of data. The format is represented below:
    
    [ 0x02 ]  [ variable ]
    ---------------------
    | Length |  Data... |
    ---------------------

Resources:

<https://en.wikipedia.org/wiki/Endianness>

<https://en.wikipedia.org/wiki/Integer_%28computer_science%29>

<https://docs.python.org/3/library/struct.html>

<https://docs.python.org/3/tutorial/inputoutput.html>



In [4]:
from embsec import Serial

def send_file():
    ser = Serial("/embsec/io_python/send_file")
    with open('file.bin', 'rb') as fp:
        file_contents = fp.read()
    
    filesize = len(file_contents)
    
    ser.write(struct.pack('<h' + str(filesize) + 's',filesize ,file_contents))
    return ser.read_until()

send_file()


b'embsec{send_file_34841aa0f7ee5d26}\n'

### Challenge Name: send_large_file (/embsec/io_python/send_large_file)


    1. Read 'large_file.bin' from your local system
    3. Send the data in frames to the serial device (frame format below)
    4. Send zero-length frame to indicate end of transmission

    The serial device expects that a frame begins with a little-endian short 
    indicating the size of the frame and then frame data. The maximum frame 
    size is 16 bytes. The frame format is represented below:
    
    [ 0x02 ]  [ up to 0xE bytes ]
    ----------------------------
    | Length |      Data...    |
    ----------------------------
    
Resources:
<https://en.wikipedia.org/wiki/Endianness>
<https://en.wikipedia.org/wiki/Integer_%28computer_science%29>
<https://docs.python.org/3/library/struct.html>
<https://docs.python.org/3/tutorial/inputoutput.html>
<https://pyserial.readthedocs.io/en/latest/shortintro.html>



In [8]:
from embsec import Serial
import struct

def send_large_file():
    ser = Serial("/embsec/io_python/send_large_file")
    with open('large_file.bin', 'rb') as fp:
        file_contents = fp.read()
    
    for i in range(0, len(file_contents), 8):
        file_chunk = file_contents[i: i+8]
        frame = struct.pack('<h8s', 8, file_chunk)
        ser.write(frame)
    print("h")
    return ser.read_until()
send_large_file()


h


Process SendLargeFile-8:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.7/site-packages/core/challenge.py", line 73, in run
    self.grader(*self.grader_pttys)
  File "/opt/conda/lib/python3.7/site-packages/embsec/lessons/io_python/io_python.py", line 145, in send_large_file_grader
    frame_size, = struct.unpack("<H", ser.read(2))
  File "/opt/conda/lib/python3.7/site-packages/core/pseudo_serial.py", line 111, in read
    d = self._inq.get(timeout=timeout)
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 94, in get
    res = self._recv_bytes()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 3

KeyboardInterrupt: 